In [1]:

%load_ext autoreload
%autoreload 2



In [2]:
import os; os.chdir(f'{os.environ["MT_ROOT"]}/src/')
import torch as th
from torch import nn
import torchvision

import constants as ct
from models.i3d import I3D
from models.ae_i3d import AEI3D
from models.vae_i3d import VAEI3D
from models.tarn import TimeAlignedResNet
from models.helpers import count_parameters

In [3]:
class A:
    x = None
    
    def __init__(self):
        self.x = nn.ConstantPad3d(1, 0)
class B(nn.Module):
    x = None
    def __init__(self):
        super(B, self).__init__()
        self.x = nn.ConstantPad3d(1, 0)

In [4]:
a = A()
b = B()
a.x, b.x

(ConstantPad3d(padding=(1, 1, 1, 1, 1, 1), value=0), None)

In [5]:
s = nn.Softmax(dim=-1)
x = th.randn((2, 4, 10))
a = s(x).mean(dim=1)
b = s(x.mean(dim=1))
a.shape, b.shape, th.argmax(a), a, th.argmax(b), b

(torch.Size([2, 10]),
 torch.Size([2, 10]),
 tensor(15),
 tensor([[0.0933, 0.1081, 0.1573, 0.0366, 0.0571, 0.1469, 0.0900, 0.1131, 0.1029,
          0.0948],
         [0.1145, 0.0731, 0.1331, 0.1205, 0.0755, 0.1574, 0.1127, 0.0948, 0.0589,
          0.0597]]),
 tensor(15),
 tensor([[0.1021, 0.0978, 0.1601, 0.0480, 0.0556, 0.1546, 0.0832, 0.1032, 0.0854,
          0.1099],
         [0.0836, 0.0811, 0.1441, 0.1451, 0.0771, 0.1838, 0.0962, 0.0921, 0.0467,
          0.0502]]))

In [6]:
i3d = I3D(ct.SMTH_NUM_CLASSES, 0.0)
tarn = TimeAlignedResNet(4, 0.0, ct.SMTH_NUM_CLASSES)
aei3d = AEI3D(1024, 0.0, ct.SMTH_NUM_CLASSES)
vaei3d = VAEI3D(1024, 0.0, ct.SMTH_NUM_CLASSES, 'soft')
le_net = torchvision.models.GoogLeNet(num_classes=ct.SMTH_NUM_CLASSES, aux_logits=False)
inception = torchvision.models.inception_v3(num_classes=ct.SMTH_NUM_CLASSES)
res_net = torchvision.models.resnet18(num_classes=ct.SMTH_NUM_CLASSES)
print(f'LeNet:\t\t{count_parameters(le_net)}')
print(f'Inception:\t{count_parameters(inception)}')
print(f'ResNet18:\t{count_parameters(res_net)}')
print(f'I3D:\t\t{count_parameters(i3d)}')
print(f'TARN:\t\t{count_parameters(tarn)}')
print(f'AEI3D:\t\t{count_parameters(aei3d)}')
print(f'VAEI3D:\t\t{count_parameters(vaei3d)}')

TypeError: __init__() takes 1 positional argument but 2 were given

In [67]:

time_steps = 2
batch_size = 3
samples = 4
embed_size = 5
z = [
    th.tensor(range(0, 60)).reshape(batch_size, samples, embed_size),
    th.tensor(range(60, 120)).reshape(batch_size, samples, embed_size),
]
z


[tensor([[[ 0,  1,  2,  3,  4],
          [ 5,  6,  7,  8,  9],
          [10, 11, 12, 13, 14],
          [15, 16, 17, 18, 19]],
 
         [[20, 21, 22, 23, 24],
          [25, 26, 27, 28, 29],
          [30, 31, 32, 33, 34],
          [35, 36, 37, 38, 39]],
 
         [[40, 41, 42, 43, 44],
          [45, 46, 47, 48, 49],
          [50, 51, 52, 53, 54],
          [55, 56, 57, 58, 59]]]), tensor([[[ 60,  61,  62,  63,  64],
          [ 65,  66,  67,  68,  69],
          [ 70,  71,  72,  73,  74],
          [ 75,  76,  77,  78,  79]],
 
         [[ 80,  81,  82,  83,  84],
          [ 85,  86,  87,  88,  89],
          [ 90,  91,  92,  93,  94],
          [ 95,  96,  97,  98,  99]],
 
         [[100, 101, 102, 103, 104],
          [105, 106, 107, 108, 109],
          [110, 111, 112, 113, 114],
          [115, 116, 117, 118, 119]]])]

In [68]:

z_cat = th.cat(z, dim=0)
z_cat, z_cat.shape


(tensor([[[  0,   1,   2,   3,   4],
          [  5,   6,   7,   8,   9],
          [ 10,  11,  12,  13,  14],
          [ 15,  16,  17,  18,  19]],
 
         [[ 20,  21,  22,  23,  24],
          [ 25,  26,  27,  28,  29],
          [ 30,  31,  32,  33,  34],
          [ 35,  36,  37,  38,  39]],
 
         [[ 40,  41,  42,  43,  44],
          [ 45,  46,  47,  48,  49],
          [ 50,  51,  52,  53,  54],
          [ 55,  56,  57,  58,  59]],
 
         [[ 60,  61,  62,  63,  64],
          [ 65,  66,  67,  68,  69],
          [ 70,  71,  72,  73,  74],
          [ 75,  76,  77,  78,  79]],
 
         [[ 80,  81,  82,  83,  84],
          [ 85,  86,  87,  88,  89],
          [ 90,  91,  92,  93,  94],
          [ 95,  96,  97,  98,  99]],
 
         [[100, 101, 102, 103, 104],
          [105, 106, 107, 108, 109],
          [110, 111, 112, 113, 114],
          [115, 116, 117, 118, 119]]]), torch.Size([6, 4, 5]))

In [69]:

z_cat = z_cat.reshape(time_steps, batch_size, samples, embed_size)
z_cat, z_cat.shape


(tensor([[[[  0,   1,   2,   3,   4],
           [  5,   6,   7,   8,   9],
           [ 10,  11,  12,  13,  14],
           [ 15,  16,  17,  18,  19]],
 
          [[ 20,  21,  22,  23,  24],
           [ 25,  26,  27,  28,  29],
           [ 30,  31,  32,  33,  34],
           [ 35,  36,  37,  38,  39]],
 
          [[ 40,  41,  42,  43,  44],
           [ 45,  46,  47,  48,  49],
           [ 50,  51,  52,  53,  54],
           [ 55,  56,  57,  58,  59]]],
 
 
         [[[ 60,  61,  62,  63,  64],
           [ 65,  66,  67,  68,  69],
           [ 70,  71,  72,  73,  74],
           [ 75,  76,  77,  78,  79]],
 
          [[ 80,  81,  82,  83,  84],
           [ 85,  86,  87,  88,  89],
           [ 90,  91,  92,  93,  94],
           [ 95,  96,  97,  98,  99]],
 
          [[100, 101, 102, 103, 104],
           [105, 106, 107, 108, 109],
           [110, 111, 112, 113, 114],
           [115, 116, 117, 118, 119]]]]), torch.Size([2, 3, 4, 5]))

In [70]:

z_cat = z_cat.transpose(0, 1).transpose(1, 2)
z_cat, z_cat.shape


(tensor([[[[  0,   1,   2,   3,   4],
           [ 60,  61,  62,  63,  64]],
 
          [[  5,   6,   7,   8,   9],
           [ 65,  66,  67,  68,  69]],
 
          [[ 10,  11,  12,  13,  14],
           [ 70,  71,  72,  73,  74]],
 
          [[ 15,  16,  17,  18,  19],
           [ 75,  76,  77,  78,  79]]],
 
 
         [[[ 20,  21,  22,  23,  24],
           [ 80,  81,  82,  83,  84]],
 
          [[ 25,  26,  27,  28,  29],
           [ 85,  86,  87,  88,  89]],
 
          [[ 30,  31,  32,  33,  34],
           [ 90,  91,  92,  93,  94]],
 
          [[ 35,  36,  37,  38,  39],
           [ 95,  96,  97,  98,  99]]],
 
 
         [[[ 40,  41,  42,  43,  44],
           [100, 101, 102, 103, 104]],
 
          [[ 45,  46,  47,  48,  49],
           [105, 106, 107, 108, 109]],
 
          [[ 50,  51,  52,  53,  54],
           [110, 111, 112, 113, 114]],
 
          [[ 55,  56,  57,  58,  59],
           [115, 116, 117, 118, 119]]]]), torch.Size([3, 4, 2, 5]))

In [61]:

bs, ns, t, e = z_cat.shape
z_cat = z_cat.reshape(bs * ns, t * e)
z_cat, z_cat.shape


(tensor([[  0,   1,   2,   3,   4,  60,  61,  62,  63,  64],
         [  5,   6,   7,   8,   9,  65,  66,  67,  68,  69],
         [ 10,  11,  12,  13,  14,  70,  71,  72,  73,  74],
         [ 15,  16,  17,  18,  19,  75,  76,  77,  78,  79],
         [ 20,  21,  22,  23,  24,  80,  81,  82,  83,  84],
         [ 25,  26,  27,  28,  29,  85,  86,  87,  88,  89],
         [ 30,  31,  32,  33,  34,  90,  91,  92,  93,  94],
         [ 35,  36,  37,  38,  39,  95,  96,  97,  98,  99],
         [ 40,  41,  42,  43,  44, 100, 101, 102, 103, 104],
         [ 45,  46,  47,  48,  49, 105, 106, 107, 108, 109],
         [ 50,  51,  52,  53,  54, 110, 111, 112, 113, 114],
         [ 55,  56,  57,  58,  59, 115, 116, 117, 118, 119]]),
 torch.Size([12, 10]))

In [62]:

z_cat.reshape(bs, ns, t * e)


tensor([[[  0,   1,   2,   3,   4,  60,  61,  62,  63,  64],
         [  5,   6,   7,   8,   9,  65,  66,  67,  68,  69],
         [ 10,  11,  12,  13,  14,  70,  71,  72,  73,  74],
         [ 15,  16,  17,  18,  19,  75,  76,  77,  78,  79]],

        [[ 20,  21,  22,  23,  24,  80,  81,  82,  83,  84],
         [ 25,  26,  27,  28,  29,  85,  86,  87,  88,  89],
         [ 30,  31,  32,  33,  34,  90,  91,  92,  93,  94],
         [ 35,  36,  37,  38,  39,  95,  96,  97,  98,  99]],

        [[ 40,  41,  42,  43,  44, 100, 101, 102, 103, 104],
         [ 45,  46,  47,  48,  49, 105, 106, 107, 108, 109],
         [ 50,  51,  52,  53,  54, 110, 111, 112, 113, 114],
         [ 55,  56,  57,  58,  59, 115, 116, 117, 118, 119]]])